In [22]:
import tensorflow as tf
from tensorflow import keras

class ConvResidualBlock(keras.layers.Layer):

    def __init__(self, kernel_size, filters, padding, activation, seed):
        super().__init__()
        initializer = keras.initializers.HeUniform(seed=seed)
        self.conv_1 = keras.layers.Conv2D(kernel_size, filters[0], padding=padding, kernel_initializer=initializer)
        self.conv_2 = keras.layers.Conv2D(kernel_size, filters[1], padding=padding, kernel_initializer=initializer)
        self.activation = keras.layers.Activation(activation)


    def call(self, inputs):
        x = self.conv_1(inputs)
        x = self.activation(x)
        x = self.conv_2(x)
        x = x + inputs
        return self.activation(x)

    def get_config(self):
        return {
            'name' : self._name,
            'conv_1' : self.conv_1.get_config(),
            'conv_2' : self.conv_2.get_config(),
            'shared_activation' : self.activation.get_config()
        }


class ConvResidualBatchNormBlock(ConvResidualBlock):

    def __init__(self, kernel_size, filters, padding, activation, seed):
        super().__init__(kernel_size, filters, padding, activation, seed)
        self.batch_norm_1 = keras.layers.BatchNormalization()
        self.batch_norm_2 = keras.layers.BatchNormalization()


    def call(self, inputs):
        x = self.conv_1(inputs)
        x = self.batch_norm_1(x)
        x = self.activation(x)
        x = self.conv_2(x)
        x = self.batch_norm_2(x)
        x = x + inputs
        return self.activation(x)

    def get_config(self):
        return {
            **super().get_config(),
            'batch_norm_1' : self.batch_norm_1.get_config(),
            'batch_norm_2' : self.batch_norm_2.get_config()
        }



layer = ConvResidualBatchNormBlock(3, [32, 64], 'same', 'relu', 1412)
inp = keras.Input((256, 256, 3))

model = keras.Model(inp, layer(inp))

t = tf.constant(1.0, shape=(1, 256, 256, 3))
model(t)

print(model.summary(expand_nested=True))

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 conv_residual_batch_norm_bl  (None, 256, 256, 3)      46110     
 ock_6 (ConvResidualBatchNor                                     
 mBlock)                                                         
                                                                 
Total params: 46,110
Trainable params: 46,098
Non-trainable params: 12
_________________________________________________________________
None


In [ ]:
from tensorflow import keras

def create_conv_batch_norm_residual_block(kernel_size, filters, padding_type, activation, kernel_initializer):
    input = keras.Input(shape=(256, 256, 3))
    x = keras.layers.Conv2D(filters[0], kernel_size, padding=padding_type, activation=activation, kernel_initializer=kernel_initializer)(input)
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Activation(activation)(x)
    x = keras.layers.Conv2D(3, kernel_size, padding=padding_type, activation=activation, kernel_initializer=kernel_initializer)(x)
    x = keras.layers.BatchNormalization()(x)
    x = x + input
    output = keras.layers.Activation(activation)(x)
    return keras.Model(input, output)


input = keras.Input(shape=(256, 256, 3))
submodel = create_conv_batch_norm_residual_block(3, [32, 32], 'same', 'relu', keras.initializers.GlorotNormal())
model = keras.Model(input, submodel(input))
model.summary(expand_nested=True)

In [21]:
from utils.file_io_utils import *
from utils.paths_utils import *

write_json_string(model.to_json(), join_path('./', 'model_architecture.json'))

In [18]:
layer._name

'conv_residual_batch_norm_block_4'

In [13]:
y = {'d' : {'f' : 4}}
x = {'a' : {'b': 3}, 'c' : 3, **y}

In [14]:
print(x)

{'a': {'b': 3}, 'c': 3, 'd': {'f': 4}}
